In [24]:
import pandas as pd 
import numpy as np 
import glob
import matplotlib.pyplot as plt
from time import time

from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, recall_score
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold

# Differnt types of Naive Bayes Classifiers
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../twitter_data_train_multiclass.csv')
test = pd.read_csv('../twitter_data_test_multiclass.csv')

In [3]:
df.head()

,Unnamed: 0,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,default_profile,default_profile_image,geo_enabled,profile_image_url,profile_use_background_image,profile_background_tile,protected,verified,description,account_type,tweets_list,tweet_frequency,number_of_tags,number_of_mentions,tweet_weekend_frequency,tweet_weekday_frequency,following_to_followers_ratio,username_length,screen_name_length,username_spec_char_count,screen_name_spec_char_count,has_url,tweets_list_processed,description_processed,tweets_0,tweets_1,tweets_2,tweets_3,tweets_4,tweets_5,tweets_6,tweets_7,tweets_8,tweets_9,tweets_10,tweets_11,tweets_12,tweets_13,tweets_14,tweets_15,tweets_16,tweets_17,tweets_18,tweets_19,tweets_20,tweets_21,tweets_22,tweets_23,tweets_24,tweets_25,tweets_26,tweets_27,tweets_28,tweets_29,tweets_30,tweets_31,tweets_32,tweets_33,tweets_34,tweets_35,tweets_36,tweets_37,tweets_38,tweets_39,tweets_40,tweets_41,tweets_42,tweets_43,tweets_44,tweets_45,tweets_46,tweets_47,tweets_48,tweets_49,tweets_50,tweets_51,tweets_52,tweets_53,tweets_54,tweets_55,tweets_56,tweets_57,tweets_58,tweets_59,tweets_60,tweets_61,tweets_62,tweets_63,tweets_64,tweets_65,tweets_66,tweets_67,tweets_68,tweets_69,tweets_70,tweets_71,tweets_72,tweets_73,tweets_74,tweets_75,tweets_76,tweets_77,tweets_78,tweets_79,tweets_80,tweets_81,tweets_82,tweets_83,tweets_84,tweets_85,tweets_86,tweets_87,tweets_88,tweets_89,tweets_90,tweets_91,tweets_92,tweets_93,tweets_94,tweets_95,tweets_96,tweets_97,tweets_98,tweets_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,account_type_multi
0,0,28088585,Scott Sandler,DigiRanger1994,124434,1818,1985,7452,41,NaN,0.0,0.0,1.0,http://pbs.twimg.com/profile_images/5833438974...,1.0,1.0,NaN,NaN,"i watch a lot of TV, movies and webshows, play...",1,NaN,0.000000,0.0,0.0,0.000000,0.000000,1.091859,13,14,1,0,0,NaN,i watch a lot of TV movies and webshows play a...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [21]:
df.columns

Index(['Unnamed: 0', 'id', 'name', 'screen_name', 'statuses_count',
       'followers_count', 'friends_count', 'favourites_count', 'listed_count',
       'url',
       ...
       'description_91', 'description_92', 'description_93', 'description_94',
       'description_95', 'description_96', 'description_97', 'description_98',
       'description_99', 'account_type_multi'],
      dtype='object', length=235)

## Get to the correct data format

In [4]:
to_drop = ['Unnamed: 0', 'screen_name', 'url', 'profile_image_url', 'description',
           'id', 'name', 'account_type', 'tweets_list', 'tweets_list_processed',
          'description_processed', 'protected', 'verified', 'account_type_multi']

X_train, y_train = df.drop(to_drop, axis=1), df['account_type_multi']

In [5]:
X_test, y_test = test.drop(to_drop, axis=1), test['account_type_multi']

In [6]:
X_train.head()


,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,geo_enabled,profile_use_background_image,profile_background_tile,tweet_frequency,number_of_tags,number_of_mentions,tweet_weekend_frequency,tweet_weekday_frequency,following_to_followers_ratio,username_length,screen_name_length,username_spec_char_count,screen_name_spec_char_count,has_url,tweets_0,tweets_1,tweets_2,tweets_3,tweets_4,tweets_5,tweets_6,tweets_7,tweets_8,tweets_9,tweets_10,tweets_11,tweets_12,tweets_13,tweets_14,tweets_15,tweets_16,tweets_17,tweets_18,tweets_19,tweets_20,tweets_21,tweets_22,tweets_23,tweets_24,tweets_25,tweets_26,tweets_27,tweets_28,tweets_29,tweets_30,tweets_31,tweets_32,tweets_33,tweets_34,tweets_35,tweets_36,tweets_37,tweets_38,tweets_39,tweets_40,tweets_41,tweets_42,tweets_43,tweets_44,tweets_45,tweets_46,tweets_47,tweets_48,tweets_49,tweets_50,tweets_51,tweets_52,tweets_53,tweets_54,tweets_55,tweets_56,tweets_57,tweets_58,tweets_59,tweets_60,tweets_61,tweets_62,tweets_63,tweets_64,tweets_65,tweets_66,tweets_67,tweets_68,tweets_69,tweets_70,tweets_71,tweets_72,tweets_73,tweets_74,tweets_75,tweets_76,tweets_77,tweets_78,tweets_79,tweets_80,tweets_81,tweets_82,tweets_83,tweets_84,tweets_85,tweets_86,tweets_87,tweets_88,tweets_89,tweets_90,tweets_91,tweets_92,tweets_93,tweets_94,tweets_95,tweets_96,tweets_97,tweets_98,tweets_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99
0,124434,1818,1985,7452,41,0.0,0.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.000000,0.000000,1.091859,13,14,1,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

## Mixed NB Approach 1 
train a gaussian NB model on all features

In [7]:
model_G = GaussianNB()
clf_G = model_G.fit(X_train, y_train)
y_pred = model_G.predict(X_test)

In [46]:
# train classification report
y_train_pred = model_G.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.75      0.43      0.55      2343
           1       0.47      0.97      0.63      2701
           2       0.14      0.36      0.20       300
           3       0.82      0.06      0.12      3706
           4       0.18      0.62      0.28       316
           5       0.22      0.99      0.36        79

    accuracy                           0.45      9445
   macro avg       0.43      0.57      0.36      9445
weighted avg       0.65      0.45      0.38      9445



In [47]:
# test classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.38      0.50       413
           1       0.46      0.97      0.62       463
           2       0.16      0.41      0.23        59
           3       0.76      0.06      0.10       663
           4       0.16      0.65      0.26        51
           5       0.26      1.00      0.41        18

    accuracy                           0.43      1667
   macro avg       0.42      0.58      0.36      1667
weighted avg       0.62      0.43      0.36      1667



## Mixed NB Approach 2
train three separate models using continuous, binary and categorical variables and then train the final model based on predictions from the first three models

In [11]:
binary_cols = ['default_profile', 'default_profile_image', 'geo_enabled', 'default_profile_image', 'profile_use_background_image',                          'profile_background_tile', 'has_url']
continuous_cols = ['statuses_count', 'followers_count', 'friends_count', 'favourites_count', 'listed_count', 'tweet_frequency',    
                    'number_of_tags', 'number_of_mentions', 'tweet_weekend_frequency', 'tweet_weekday_frequency',               
                    'following_to_followers_ratio', 'username_length', 'screen_name_length', 'username_spec_char_count', 
                    'screen_name_spec_char_count']												

In [12]:
train_binary = X_train[binary_cols]
train_continuous = X_train[continuous_cols]
train_text = X_train.drop(binary_cols+continuous_cols, axis=1) 

In [13]:
test_binary = X_test[binary_cols]
test_continuous = X_test[continuous_cols]
test_text = X_test.drop(binary_cols+continuous_cols, axis=1) 

In [14]:
# ----- Fit three models -----

model_G = GaussianNB()
clf_G = model_G.fit(train_continuous, y_train)

model_M = MultinomialNB()
clf_M = model_M.fit(train_text, y_train)

model_B = BernoulliNB()
clf_B = model_B.fit(train_binary, y_train)

# ----- Get probability predictions from each model -----
# On training data
G_train_probas = model_G.predict_proba(train_continuous)
M_train_probas = model_M.predict_proba(train_text)
B_train_probas = model_B.predict_proba(train_binary)

# And on testing data
G_test_probas = model_G.predict_proba(test_continuous)
M_test_probas = model_M.predict_proba(test_text)
B_test_probas = model_B.predict_proba(test_binary)

# Combine probability prediction for class=1 from both models
X_new_train = np.c_[(G_train_probas[:,1], M_train_probas[:,1], B_train_probas[:,1])] # Train
X_new_test = np.c_[(G_test_probas[:,1], M_test_probas[:,1], B_test_probas[:,1])] # Test


# ----- Fit Gaussian model on the X_new -----
model = GaussianNB()
clf = model.fit(X_new_train, y_train)
y_pred = model.predict(X_new_test)

In [52]:
# train classification report
y_train_pred = model.predict(X_new_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.76      0.30      0.43      2343
           1       0.98      0.91      0.95      2701
           2       0.33      0.30      0.31       300
           3       0.76      0.63      0.69      3706
           4       0.26      0.98      0.42       316
           5       0.05      0.96      0.10        79

    accuracy                           0.63      9445
   macro avg       0.52      0.68      0.48      9445
weighted avg       0.79      0.63      0.67      9445



In [53]:
# test classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.30      0.43       413
           1       0.98      0.89      0.93       463
           2       0.30      0.29      0.29        59
           3       0.76      0.62      0.68       663
           4       0.24      0.94      0.38        51
           5       0.06      0.94      0.12        18

    accuracy                           0.62      1667
   macro avg       0.51      0.66      0.47      1667
weighted avg       0.77      0.62      0.66      1667



## Hyperparameter Tuning

In [10]:
param_grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)

For approach 1:

In [15]:
model_grid1 = RandomizedSearchCV(estimator = GaussianNB(), 
                            param_distributions = param_grid, 
                            verbose = 1, 
                            cv = cv_method,
                            scoring = 'f1_weighted')
model_grid1.fit(X_train, y_train)
print('--------------------------------------------------------')
print('Best Estimator: ', model_grid1.best_estimator_)

Fitting 15 folds for each of 10 candidates, totalling 150 fits
--------------------------------------------------------
Best Estimator:  GaussianNB(var_smoothing=6.579332246575682e-07)


In [16]:
# train classification report
y_train_pred = model_grid1.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.93      0.44      0.60      2343
           1       0.78      0.94      0.85      2701
           2       0.04      0.07      0.05       300
           3       0.67      0.10      0.18      3706
           4       0.15      0.56      0.24       316
           5       0.03      0.99      0.06        79

    accuracy                           0.45      9445
   macro avg       0.43      0.52      0.33      9445
weighted avg       0.72      0.45      0.47      9445



In [17]:
# test classification report
y_pred = model_grid1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.40      0.56       413
           1       0.79      0.95      0.86       463
           2       0.05      0.10      0.07        59
           3       0.62      0.10      0.17       663
           4       0.13      0.57      0.21        51
           5       0.04      1.00      0.07        18

    accuracy                           0.43      1667
   macro avg       0.42      0.52      0.32      1667
weighted avg       0.70      0.43      0.46      1667



For approach 2:

In [18]:
model_grid2 = RandomizedSearchCV(estimator = GaussianNB(), 
                            param_distributions = param_grid, 
                            verbose = 1, 
                            cv = cv_method,
                            scoring = 'f1_weighted')
model_grid2.fit(X_new_train, y_train)
print('--------------------------------------------------------')
print('Best Estimator: ', model_grid2.best_estimator_)

Fitting 15 folds for each of 10 candidates, totalling 150 fits
--------------------------------------------------------
Best Estimator:  GaussianNB(var_smoothing=0.04328761281083057)


In [19]:
# train classification report
y_train_pred = model_grid2.predict(X_new_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.80      0.42      0.55      2343
           1       0.96      0.91      0.93      2701
           2       0.42      0.29      0.34       300
           3       0.68      0.94      0.79      3706
           4       0.33      0.30      0.32       316
           5       0.00      0.00      0.00        79

    accuracy                           0.75      9445
   macro avg       0.53      0.48      0.49      9445
weighted avg       0.76      0.75      0.74      9445



In [20]:
# test classification report
y_pred = model_grid2.predict(X_new_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.43      0.55       413
           1       0.95      0.89      0.92       463
           2       0.38      0.27      0.32        59
           3       0.68      0.93      0.79       663
           4       0.37      0.49      0.42        51
           5       0.00      0.00      0.00        18

    accuracy                           0.75      1667
   macro avg       0.53      0.50      0.50      1667
weighted avg       0.76      0.75      0.73      1667



## Best NB Model
Approach 2 has a higher recall score than approach 1.

## Run time

In [34]:
start_time = time()
model_time = GaussianNB(var_smoothing=0.04328761281083057)
clf_best = model_time.fit(X_new_train, y_train)
y_pred = model_time.predict(X_new_test)
run_time = time() - start_time
print("Run time: ", run_time)

Run time:  0.11637187004089355


## Feature importance

In [35]:
imps = permutation_importance(model_time, X_test, y_test)

In [36]:
imps_sorted = imps.importances_mean.argsort()[::-1]

print(np.take(X_train.columns, imps_sorted[: 20]))

Index(['statuses_count', 'favourites_count', 'friends_count', 'tweets_49',
       'tweets_60', 'tweets_59', 'tweets_58', 'tweets_57', 'tweets_56',
       'tweets_55', 'tweets_54', 'tweets_53', 'tweets_52', 'tweets_51',
       'tweets_50', 'tweets_48', 'tweets_62', 'tweets_47', 'tweets_46',
       'tweets_45'],
      dtype='object')
